In [1]:
#!pip install --upgrade plotly
#!pip install --upgrade ipywidgets
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
import ipywidgets as widgets
widgets.IntSlider()

IntSlider(value=0)

In [2]:
import open3d as o3d
import numpy as np
import plotly.graph_objs as go
from ipywidgets import Output, VBox, HBox, Text, Button
from IPython.display import display
import os

# Load the textured point cloud
input_file_path = r"C:\Users\wenru\Desktop\POLYCAM\Tall Oak (Burnam Beeches)-pc-poly\point_cloud.ply"
pcd = o3d.io.read_point_cloud(input_file_path)

# Print original point cloud information
print("Original point cloud:")
print(f"Number of points: {len(pcd.points)}")

# Perform voxel downsampling
voxel_size = 0.05  # Adjust voxel size as needed
downsampled_pcd = pcd.voxel_down_sample(voxel_size)

# Print downsampled point cloud information
print("Downsampled point cloud:")
print(f"Number of points: {len(downsampled_pcd.points)}")

# Ensure color information is correctly loaded
if not downsampled_pcd.has_colors():
    print("Point cloud does not have color information!")
else:
    print("Point cloud has color information!")

# Convert the downsampled point cloud data for 2D display
points = np.asarray(downsampled_pcd.points)
colors = np.asarray(downsampled_pcd.colors)

# Set the lowest point to Z=0
min_z = np.min(points[:, 2])
points[:, 2] -= min_z

# Convert color values from [0, 1] range to [0, 255] range
colors = (colors * 255).astype(np.uint8)

# Convert color values to RGB string format for proper display in Plotly
colors_rgb = ['rgb({}, {}, {})'.format(r, g, b) for r, g, b in colors]

# Visualize the point cloud data using Plotly and display colors
scatter = go.Scatter3d(
    x=points[:, 0],
    y=points[:, 1],
    z=points[:, 2],
    mode='markers',
    marker=dict(size=2, color=colors_rgb, opacity=0.8)
)

layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    ),
    width=1000,  # Set width
    height=800  # Set height
)

fig = go.FigureWidget(data=[scatter], layout=layout)

# Output box to display interaction information
output = Output()

# Store clicked point information
clicked_points = []
point_names = {}
labelled_indices = []

# Get the input file directory and set the output file path
output_folder = os.path.dirname(input_file_path)
output_file = os.path.join(output_folder, "point_labels.txt")

# If the file exists, clear its content
if os.path.exists(output_file):
    os.remove(output_file)

# Handle click events
def on_click(trace, points, state):
    """
    Handles the click event on the Plotly 3D plot. When a point is clicked, it is stored,
    and the user is prompted to enter a name for that point.

    Args:
        trace: The trace from the Plotly plot that was clicked.
        points: The points object containing information about the clicked point(s).
        state: The state of the plot at the time of the click.
    """
    with output:
        for i in points.point_inds:
            coord = (trace.x[i], trace.y[i], trace.z[i])
            output.append_stdout(f"Point clicked: {coord}\n")
            clicked_points.append(coord)
            labelled_indices.append(i)  # Store the index of the labeled point
            # Display text input and buttons for the user to name the point
            point_index = len(clicked_points)
            name_input = Text(description=f"Point {point_index}:")
            submit_button = Button(description="Submit")
            cancel_button = Button(description="Cancel")
            hbox = HBox([name_input, submit_button, cancel_button])
            display(hbox)

            def submit_name(b):
                save_point_name(point_index, name_input.value, coord)
                hbox.close()

            def cancel_name(b):
                clicked_points.pop()
                labelled_indices.pop()  # Remove the index of the canceled point
                hbox.close()
                output.append_stdout(f"Point {point_index} selection cancelled.\n")

            submit_button.on_click(submit_name)
            cancel_button.on_click(cancel_name)

def save_point_name(index, name, coord):
    """
    Saves the name of a clicked point and displays it on the plot.

    Args:
        index (int): The index of the point.
        name (str): The name assigned to the point by the user.
        coord (tuple): The coordinates of the point.
    """
    with output:
        point_names[index] = name
        output.append_stdout(f"Point {index} named as: {name}\n")
        # Write point information to the file
        with open(output_file, 'a') as f:
            f.write(f"Point {index}: {coord}, Name: {name}\n")
        # Add a label to the point on the visualization
        fig.add_trace(go.Scatter3d(
            x=[coord[0]], y=[coord[1]], z=[coord[2]],
            mode='text',
            text=[name],
            textposition="top center"
        ))

# Ensure click event is only registered once
if not hasattr(fig.data[0], '_click_registered'):
    fig.data[0].on_click(on_click)
    fig.data[0]._click_registered = True

# Display Plotly figure and output box
display(VBox([fig, output]))

# Saving the labeled point cloud
def save_labeled_point_cloud():
    """
    Saves the labeled point cloud data to a file, including the labels for each selected point.
    The labels are stored in a separate text file as Open3D does not support string attributes directly.
    """
    labels = np.array([''] * len(points), dtype=object)
    for idx, label in zip(labelled_indices, point_names.values()):
        labels[idx] = label
    
    # Add labels as a scalar field (or custom attribute)
    labeled_pcd = o3d.geometry.PointCloud()
    labeled_pcd.points = o3d.utility.Vector3dVector(points)
    labeled_pcd.colors = o3d.utility.Vector3dVector(colors / 255.0)  # Normalize colors back to [0, 1]
    
    # Here we save labels in a text format, as Open3D does not natively support string attributes
    np.savetxt(os.path.join(output_folder, "point_cloud_labels.txt"), labels, fmt="%s")
    
    # Save the downsampled point cloud
    labeled_ply_file = os.path.join(output_folder, "labeled_point_cloud.ply")
    o3d.io.write_point_cloud(labeled_ply_file, labeled_pcd)
    print(f"Labeled point cloud saved to {labeled_ply_file}")

# Button to trigger saving the labeled point cloud
save_button = Button(description="Save Labeled Point Cloud")
save_button.on_click(lambda b: save_labeled_point_cloud())
display(save_button)

# Debug information
print("Visualization setup complete.")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Original point cloud:
Number of points: 10796580
Downsampled point cloud:
Number of points: 14909
Point cloud has color information!


    'data': [{'marker': {'color': [rgb(51, 73, 47), rgb(77, 86, 76), rgb(69, 66,…

Button(description='Save Labeled Point Cloud', style=ButtonStyle())

Visualization setup complete.
